# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd_ai 0.5.1
Uninstalling todd_ai-0.5.1:
  Successfully uninstalled todd_ai-0.5.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

## Preparation

### Models

In [3]:
@todd.registries.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.registries.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    logger=dict(),
    callbacks=[],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-06-26 16:57:13,752 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:13,755 25854:140704458489472][base.py:67 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



zsh:1: command not found: tree


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[dict(type='LogCallback', interval=5)],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-06-26 16:57:15,523 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:15,528 25854:140704458489472][base.py:67 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:15,536 25854:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-26 16:57:15,540 25854:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-26 16:57:15,544 25854:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-06-26 16:57:15,549 25854:140704458489472][log.py:87 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20])


zsh:1: command not found: tree


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-26 16:57:17,365 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:17,370 25854:140704458489472][base.py:67 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:17,370 25854:140704458489472][base.py:67 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:17,379 25854:140704458489472][log.py:87 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([9, 3]), 'y': tensor([18,  6])} weight=0.000 loss=12.000
[2024-06-26 16:57:17,382 25854:140704458489472][log.py:87 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([10,  1]), 'y': tensor([20,  2])} weight=0.000 loss=11.000
[2024-06-26 16:57:17,385 25854:140704458489472][log.py:87 todd.IterBasedTrainer.iter_based_trainer after_run

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-26 16:57:17,424 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:17,428 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:17,431 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-06-26 16:57:17,428 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:17,431 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-06-26 16:57:17,436 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([ 5, 10]), 'y': tensor([10, 20])} weight=0.000 loss=15.000
[2024-06-26 16:57:17,439 25854:1407

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-26 16:57:17,498 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:18,102 25854:140704458489472][log.py:49 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.10.0
todd_version: 0.5.1
cuda_home: None
git_commit_id: a530070
git_status: M tutorials/runners.ipynb
[2024-06-26 16:57:18,105 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:18,112 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-26 16:57:18,116 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2024-06-26 16:57:18,157 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:18,161 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:18,169 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-26 16:57:18,174 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-26 16:57:18,178 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-06-26 16:57:18,183 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x


zsh:1: command not found: tree

[2024-06-26 16:57:18,161 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:18,169 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-26 16:57:18,174 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-26 16:57:18,178 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-06-26 16:57:18,183 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2024-06-26 16:57:21,568 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:21,587 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:21,587 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:22,110 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-26 16:57:22,630 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-26 16:57:23,155 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:00 batch={'x': ten

In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", ema=dict(decay=0.2)),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2024-06-26 16:57:23,701 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:23,703 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:23,703 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:25,229 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:04 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-26 16:57:29,253 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:05 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-26 16:57:34,277 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:03 batch={'x': ten

In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-26 16:57:39,324 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.


[2024-06-26 16:57:39,569 25854:140704458489472][log.py:49 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.10.0
todd_version: 0.5.1
cuda_home: None
git_commit_id: a530070
git_status: M tutorials/runners.ipynb
[2024-06-26 16:57:39,570 25854:140704458489472][base.py:67 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:39,577 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-06-26 16:57:39,580 25854:140704458489472][log.py:87 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:00 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-06-26 16:57:39,570 25854:140704458489472][base

### Git

In [16]:
config = todd.Config(
    type="Validator",
    name="git_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type="GitCallback", diff='HEAD -- ":(exclude)*.ipynb"'),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )

    !echo
    !cat {work_dirs}/git_callback/*.log

[2024-06-26 16:57:39,606 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:39,735 25854:140704458489472][git.py:43 todd.Validator.git_callback init] INFO: Saving git diff to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpk7571pfo/git_callback/git_diff_2024-06-26T16-57-39_734738-08-00.log
[2024-06-26 16:57:39,738 25854:140704458489472][base.py:67 todd.Validator.git_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R


### Optimize

In [17]:
config = todd.Config(
    type="IterBasedTrainer",
    name="optimize_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

[2024-06-26 16:57:41,497 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:41,501 25854:140704458489472][base.py:67 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,506 25854:140704458489472][log.py:87 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([6, 9]), 'y': tensor([12, 18])} weight=0.000 loss=15.000
[2024-06-26 16:57:41,501 25854:140704458489472][base.py:67 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,506 25854:140704458489472][log.py:87 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([6, 9]), 'y': tensor([12, 18])} weight=0.000 loss=15.000
[2024-06-26 16:57:41,508 25854:140704458489472][log.py:87 todd.IterBasedTrainer.optimize_callback after_run_iter] 

### Learning Rate Schedule

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-26 16:57:41,539 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:41,542 25854:140704458489472][base.py:67 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,548 25854:140704458489472][log.py:87 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.000 loss=10.000 lr=['1.667e-03']
[2024-06-26 16:57:41,542 25854:140704458489472][base.py:67 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,548 25854:140704458489472][log.py:87 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.000 loss=10.000 lr=['1.667e-03']
[2024-06-26 16:57:41,550 25854:140704458489472][log.py:87 todd.IterB

In [19]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=4),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-26 16:57:41,582 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:41,585 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,587 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-06-26 16:57:41,585 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,587 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-06-26 16:57:41,592 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/10] batch={'x': tensor([4, 2]), 'y': tensor([8, 4])} weight=0.000 loss=6.000 lr=['1.667e-03']
[2024-06-26 16:57:

### Learning Rate Scale

In [20]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-06-26 16:57:41,637 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:41,639 25854:140704458489472][lr.py:95 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-06-26 16:57:41,640 25854:140704458489472][base.py:67 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,645 25854:140704458489472][log.py:87 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([5, 7]), 'y': tensor([10, 14])} weight=0.000 loss=12.000
[2024-06-26 16:57:41,639 25854:140704458489472][lr.py:95 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-06-26 16:57:41,640 25854:140704458489472][base.py:67 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R

### Checkpoint

In [21]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2024-06-26 16:57:41,686 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:41,689 25854:140704458489472][base.py:67 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:41,694 25854:140704458489472][log.py:87 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([ 3, 10]), 'y': tensor([ 6, 20])} weight=0.000 loss=13.000
[2024-06-26 16:57:41,695 25854:140704458489472][checkpoint.py:81 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpy2gxoy40/checkpoint_callback/checkpoints/iter_1
[2024-06-26 16:57:41,701 25854:140704458489472][log.py:87 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([7, 9]), 'y': tensor([14, 18])} weight=0.032 loss=15.740
[2024-06-26 16:57:41,703 2


zsh:1: command not found: tree



[2024-06-26 16:57:44,448 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:44,451 25854:140704458489472][checkpoint.py:55 todd.IterBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpy2gxoy40/checkpoint_callback/checkpoints/iter_5
[2024-06-26 16:57:44,457 25854:140704458489472][base.py:65 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-26 16:57:44,458 25854:140704458489472][base.py:67 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:44,462 25854:140704458489472][log.py:87 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [6/8] batch={'x': tensor([9, 1]), 'y': tensor([18,  2])} weight=0.138 loss=9.312
[2024-06-26 16:57:44,463 25854:140704458489472][checkpoint.py:81 todd.IterBasedTraine

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [22]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2024-06-26 16:57:44,524 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:44,526 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:44,528 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-06-26 16:57:44,532 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([6, 2]), 'y': tensor([12,  4])} weight=0.000 loss=8.000
[2024-06-26 16:57:44,534 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([7, 5]), 'y': tensor([14, 10])} weight=0.020 loss=11.880
[2024-06-26 16:57:44,536 25854:140704458489472][checkpoint.py:81 todd.EpochBasedTrainer.checkpoint_callback _save] INFO: Saving 


zsh:1: command not found: tree



[2024-06-26 16:57:47,069 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:47,072 25854:140704458489472][checkpoint.py:55 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpphcic27h/checkpoint_callback/checkpoints/iter_8
[2024-06-26 16:57:47,077 25854:140704458489472][base.py:65 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-26 16:57:47,077 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:47,078 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [2/3]
[2024-06-26 16:57:47,083 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [9/15] batch={'x': tensor


--------------------



[2024-06-26 16:57:49,690 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:49,692 25854:140704458489472][checkpoint.py:55 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpphcic27h/checkpoint_callback/checkpoints/iter_10
[2024-06-26 16:57:49,697 25854:140704458489472][base.py:65 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-26 16:57:49,698 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:49,699 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-06-26 16:57:49,692 25854:140704458489472][checkpoint.py:55 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kk

In [23]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2024-06-26 16:57:49,753 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:49,755 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:49,756 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-06-26 16:57:49,760 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([5, 6]), 'y': tensor([10, 12])} weight=0.000 loss=11.000
[2024-06-26 16:57:49,762 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([ 2, 10]), 'y': tensor([ 4, 20])} weight=0.027 loss=11.835
[2024-06-26 16:57:49,764 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: It


zsh:1: command not found: tree



[2024-06-26 16:57:52,298 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:52,301 25854:140704458489472][checkpoint.py:55 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp6zmfeuog/checkpoint_callback/checkpoints/epoch_2
[2024-06-26 16:57:52,305 25854:140704458489472][base.py:65 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-26 16:57:52,307 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:52,308 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-06-26 16:57:52,301 25854:140704458489472][checkpoint.py:55 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kk

### Monitor

In [24]:
class CustomError(RuntimeError):
    pass

In [25]:
@todd.registries.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [26]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2024-06-26 16:57:52,443 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:52,445 25854:140704458489472][base.py:67 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:52,448 25854:140704458489472][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x148d04890>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 232, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_25854/454645826.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner



[2024-06-26 16:57:52,445 25854:140704458489472][base.py:67 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:52,448 25854:140704458489472][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x148d04890>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 232, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_25854/454645826.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [27]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2024-06-26 16:57:54,136 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:54,139 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:54,140 25854:140704458489472][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-06-26 16:57:54,143 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([9, 8]), 'y': tensor([18, 16])} weight=0.000 loss=17.000
[2024-06-26 16:57:54,146 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [2/15] batch={'x': tensor([5, 6]), 'y': tensor([10, 12])} weight=0.043 loss=10.766
[2024-06-26 16:57:54,149 25854:140704458489472][log.py:87 todd.EpochBasedTrainer.strategy_load_model_from a


--------------------



[2024-06-26 16:57:56,721 25854:140704458489472][patches.py:190 todd _build] INFO: `worker_init_fn` is recommended to be `default`, instead of `None`.
[2024-06-26 16:57:56,724 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:56,725 25854:140704458489472][base.py:80 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpeuzo5_w0/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2024-06-26 16:57:56,727 25854:140704458489472][base.py:65 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2024-06-26 16:57:56,724 25854:140704458489472][base.py:67 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-06-26 16:57:56,725 25854:140704458489472][base.py:80 todd.EpochBasedTrain

## Dry Run

In [28]:
todd.Store.DRY_RUN = True